In [1]:
import os
import subprocess
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [3]:
clean_data = spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/zhiliny/final_project4')
clean_data.count()

22/12/07 19:21:51 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


1052477

In [4]:
clean_data.limit(1)

userid,username,user,date,text,quoted_status,retweeted_status,quote_count,retweet_count,retweeted,favorited,favorite_count,location,screen_name,timestamp_ms,stdate,entities,country,country_code
1374751479848972290,loveyou&me,"{false, Wed Mar 2...",May 20,"RT @kkukstudio: ""...",null,"{null, Fri May 20...",0,0,RT,false,0,null,tangledlocks1,1.653065323661E9,20220520,"{[], null, [], []...",null,null


In [4]:
clean_data = clean_data.drop\
('follower_count','favorite_count')

In [5]:
cleandata1 = clean_data.withColumn('description',clean_data.user['description'])
cleandata1 = cleandata1.withColumn('verified_user',clean_data.user['verified']).\
                        withColumn('follower_counted',clean_data.user['followers_count']).\
                        withColumn('user_id',clean_data.user['id']).\
                        withColumn('retweetcount',cleandata1.retweeted_status['retweet_count']).\
                        withColumn('quotecount',cleandata1.retweeted_status['quote_count']).\
                        withColumn('favourite_count',cleandata1.retweeted_status['favorite_count']).\
                        withColumn('userid1',cleandata1.retweeted_status['id'])

cleandata1.limit(1)
# cleandata1.limit(1).toPandas()

userid,username,user,date,text,quoted_status,retweeted_status,quote_count,retweet_count,retweeted,favorited,location,screen_name,timestamp_ms,stdate,entities,country,country_code,description,verified_user,follower_counted,user_id,retweetcount,quotecount,favourite_count,userid1
895426136007737345,Titus Folks,"{false, Wed Aug 0...",Apr 05,RT @BrandonStraka...,"{null, Tue Apr 05...","{null, Tue Apr 05...",0,0,RT,false,"Bloomington, IN",TitusFolks,1.649166236691E9,20220405,"{[], null, [], []...",null,null,I'm on a mission ...,false,909,895426136007737345,327,12,1119,1511148055088742400


##### Analyze the verified user

In [7]:
verified = cleandata1.select(['user_id', 'verified_user'])
verified1 = verified.dropDuplicates(['user_id', 'verified_user'])
verified1.groupby('verified_user').count()

verified_user,count
true,11622
false,655396


### Influntial Analysis

In [6]:
## Categorize the Twitterers' organizations by searching the keywords
## If the follower count is larger than 10000, I categorize the user as a 'social media Influncer'
government_entities=['[Gg]overnment','[Ss]tate','[Ff]ederal','[Cc]ongress','[Ss]enate','[Ss]enator',
                    '[Cc]ongressman','[Cc]ongresswoman','[Mm]ayor']
universities=['[Uu]niversit(y)?(ies)?','[Cc]olleg(e)?(es)','[Hh]igher [Ed]ucation','[Aa]cadamy','[Ii]nstitution']
schools=['[Ss]chool(s)?','[Pp]rimary [Ss]chool', '[Ss]econdary [Ss]chool','[Hh]igh [Ss]chool']
nonprofit_organizations = ['[Nn]onprofit','NGO','[Ff]oundation(s)?','[Nn]onprofit [Oo]rganization(s)?']
news_outlets=['[Nn]ewsletter(s)?','[Nn]ewspaper(s)?','[Jj]ournal(s)?','[Nn]ews']
social_media=['[Ff]acebook','[Ii]nstagram','[Ss]ocial [Mm]edia']

cleandata1 = cleandata1.withColumn('Twitterer_category',\
                    when(cleandata1.description.rlike('|'.join(government_entities)),'Government').\
                    when(cleandata1.description.rlike('|'.join(universities)),'Universities').\
                    when(cleandata1.description.rlike('|'.join(schools)),'Schools').\
                    when(cleandata1.description.rlike('|'.join(nonprofit_organizations)),'Nonprofit').\
                    when(cleandata1.description.rlike('|'.join(news_outlets)),'News').\
                    when(cleandata1.follower_counted>=10000,'Influencer').\
                    otherwise('Someone else'))

In [10]:
#Set the original tweets dataset
original = cleandata1.filter((cleandata1.retweeted_status.isNull()))


In [11]:
original.count()

275349

#### By message volume (original content)

In [12]:
original_volume=original
original_volume = original_volume.groupby('user_id','username').\
                  agg(count('user_id').alias('UserVolume')).\
                  orderBy('UserVolume', ascending=False)

In [13]:
original_volume.limit(10).toPandas()

,user_id,username,UserVolume
0,1486642599633031168,GraceSmartsBlog,1205
1,1350419912615976960,Grace Smart,931
2,916800542332112896,Graceville WxSTEM,374
3,3181683816,Abinaya Devi,303
4,1062426654855831552,Jordan Lindsey,282
5,2935926947,"Dr. Lena Gould, EdD, CRNA, FAANA, FAAN",264
6,1457884559677272076,Greg Grace,256
7,825753297470513152,Mitch Zawaski,231
8,798716358930432000,"Jonathan Gelber, MD, MS",167
9,1252202548569636865,adam (READ PINNED) #buyingcontent,162


#### Retweets

In [7]:
retweets = cleandata1.withColumn('total_engagement',col('retweetcount')+col('quotecount')+col('favourite_count')).\
                      withColumn('retweet_quote',col('retweetcount')+col('quotecount')).\
                      withColumn('engagement_rate',col('retweet_quote')/col('total_engagement'))

retweets.limit(1).toPandas()

,userid,username,user,date,text,quoted_status,retweeted_status,quote_count,retweet_count,retweeted,...,follower_counted,user_id,retweetcount,quotecount,favourite_count,userid1,Twitterer_category,total_engagement,retweet_quote,engagement_rate
0,1549130957625495553,ye,"(False, Mon Jul 18 20:36:29 +0000 2022, True, ...",Aug 29,RT @itz6ixtii: Step Sister Needed A Favor😆\njo...,None,"(None, Wed Aug 24 03:08:50 +0000 2022, [0, 140...",0,0,RT,...,26,1549130957625495553,41,0,260,1562275714329706496,Someone else,301,41,0.136213


In [8]:
retweets_table = retweets.select('user_id','userid1','stdate','username','Twitterer_category','country','retweeted_status','retweeted','retweetcount','quotecount','favourite_count','retweet_quote','total_engagement','engagement_rate')
retweets_table.limit(10)

user_id,userid1,stdate,username,Twitterer_category,country,retweeted_status,retweeted,retweetcount,quotecount,favourite_count,retweet_quote,total_engagement,engagement_rate
895426136007737345,1511148055088742400,20220405,Titus Folks,Government,null,"{null, Tue Apr 05...",RT,327,12,1119,339,1458,0.23251028806584362
1301696889297928192,null,20220405,The Q Origins Pro...,Influencer,null,null,,null,null,null,null,null,null
1440190888358014979,null,20220405,LiterallyNobody,Someone else,null,null,,null,null,null,null,null,null
1197965433334378499,1510996652722642947,20220405,Truth and fairness,Someone else,null,"{null, Mon Apr 04...",RT,112,17,380,129,509,0.25343811394891946
1498476831527297027,1509917251264299012,20220405,LoudounCountyMoms,Someone else,null,"{null, Fri Apr 01...",RT,128,17,462,145,607,0.23887973640856672
2460279166,1511175367062695939,20220405,Justice Seeker 🗽...,Influencer,null,"{null, Tue Apr 05...",RT,39,4,96,43,139,0.30935251798561153
1481564568845438976,null,20220405,USeducation Service,Someone else,null,null,,null,null,null,null,null,null
1009832042568138753,1510996652722642947,20220405,Midwestgal,Someone else,null,"{null, Mon Apr 04...",RT,113,17,380,130,510,0.2549019607843137
718426119951753216,1511338868452167680,20220405,Just Breathe 🧘🏾‍♀️,Someone else,null,"{null, Tue Apr 05...",RT,1,0,1,1,2,0.5
519833567,1511338795651764228,20220405,Angel,Someone else,null,"{null, Tue Apr 05...",RT,1,0,8,1,9,0.1111111111111111


In [16]:
influence_table = retweets.groupby('user_id').agg(avg('engagement_rate').alias('avg_influence_byid'))

In [17]:
retweets_table = retweets_table.join(influence_table,'user_id','left')

In [18]:
# retweets_table.limit(5)

In [19]:
#count_table = retweets.groupby('user_id').agg(count('user_id').alias('total_tweet'))
totalretweet = retweets.filter((retweets.retweeted == 'RT'))
totalretweet = totalretweet.groupby('user_id').agg(count('*').alias('total_retweet'))

In [20]:
totaloriginal = retweets.filter((retweets.retweeted != 'RT'))
totaloriginal = totaloriginal.groupby('user_id').agg(count('*').alias('total_original'))

In [21]:
retweets_table1 = retweets_table.join(totalretweet,'user_id','left')
retweets_table2 = retweets_table1.join(totaloriginal,'user_id','left')

In [19]:
retweets_table2

user_id,stdate,username,Twitterer_category,retweeted_status,retweeted,retweetcount,quotecount,favourite_count,retweet_quote,total_engagement,engagement_rate,avg_influence_byid,total_retweet,total_original
1549130957625495553,20220829,ye,Someone else,"{null, Wed Aug 24...",RT,41,0,260,41,301,0.1362126245847176,0.10230133515449434,6,null
945915328311283712,20220829,Casey (she/her),Someone else,"{null, Sun Aug 28...",RT,160,4,472,164,636,0.2578616352201258,0.3396912521440823,3,null
3359873302,20220829,wanna be me,Someone else,null,,null,null,null,null,null,null,null,null,1
1440197544068354054,20220829,S Bains,Someone else,"{null, Sun Aug 28...",RT,32,1,67,33,100,0.33,0.3270650356199204,4,null
1040651956757450753,20220829,"Sara Anderson, DVM",Someone else,"{null, Sun Aug 28...",RT,74,1,68,75,143,0.5244755244755245,0.5244755244755245,1,null
1386537728175710209,20220829,jojo,Someone else,"{null, Sat Aug 20...",RT,577,3,4507,580,5087,0.11401611952034597,0.09660367896826061,2,null
3585268873,20220829,PSJ,News,"{null, Sun Aug 28...",RT,75,1,70,76,146,0.5205479452054794,0.5205479452054794,1,null
1547763786559266816,20220829,lauras4Trump,Someone else,"{null, Sun Aug 28...",RT,12,0,5,12,17,0.7058823529411765,0.5407547069501556,10,null
91811276,20220829,kristinabraziel,Someone else,"{null, Sun Aug 28...",RT,3826,481,7935,4307,12242,0.3518215977781408,0.3182580049547422,5,null
1494489375488417792,20220829,Howe Dwayne,Someone else,"{null, Sun Aug 28...",RT,3828,482,7939,4310,12249,0.35186545840476774,0.3766903049599596,2,null


In [22]:
retweets_table2.fillna(0,subset = ['engagement_rate','avg_influence_byid','total_retweet','total_original'])
retweets_table3 = retweets_table2.withColumn('influencepertwitter',col('engagement_rate')*((col('total_retweet'))*1+(col('total_original'))*2))
retweets_table4 = retweets_table3.groupby('user_id','username').agg(sum('influencepertwitter').alias('influence_score'))

In [23]:
totalretweeted = retweets_table.filter(retweets_table.retweeted=='RT')
totalretweeted = totalretweeted.groupby('user_id').agg(count('*').alias('total_retweeted'))

In [24]:
result = retweets_table4.join(totalretweeted,'user_id','inner')
result1 = result.fillna(0,subset = ['influence_score'])
result2 = result1.orderBy('total_retweeted',ascending = False)

In [25]:
result2df = result2.toPandas()

In [23]:
totalretweeted1 = retweets_table.filter(retweets_table.retweeted=='RT')
totalretweeted1 = totalretweeted1.groupby('userid').agg(count('*').alias('total_retweeted'))

In [24]:
resultnew = retweets_table4.join(totalretweeted1,'user_id','inner')
result1new = resultnew.fillna(0,subset = ['influence_score'])
result2new = result1new.orderBy('total_retweeted',ascending = False)

In [25]:
result2dfnew = result2new.toPandas()

In [26]:
result2df

,user_id,username,influence_score,total_retweeted
0,1311337818518556672,Academic Opportunities,0.000000,210
1,1387204363,Fansly.com/Littlesubgirl,1360.090952,195
2,1387204363,Patreon.com/Littlesubgirl,2203.598327,195
3,1295715136141963267,Education World,0.000000,174
4,958729147408310272,James Finn,11492.967522,159
...,...,...,...,...
519660,1324815816718721025,💙Fanny💛Fannakapan💙,0.000000,1
519661,917036941631246336,Paula Bononi Wineland,0.000000,1
519662,75980645,glenda aquino,0.000000,1
519663,1136108938892587008,monique 🌻 nw: Jessica Jones s2,0.000000,1


In [41]:
resultdfshow = result2df[result2df['influence_score']!= 0]
resultdfshow.sort_values(by =['total_retweeted'],ascending = False).head(10)

,user_id,username,influence_score,total_retweeted
1,1387204363,Fansly.com/Littlesubgirl,1360.090952,195
2,1387204363,Patreon.com/Littlesubgirl,2203.598327,195
4,958729147408310272,James Finn,11492.967522,159
11,22311924,Alex 1984 vs 1776 🇺🇸,2058.681947,108
12,22311924,Gulag Inmate 93271 🇺🇸,354.922022,108
13,22311924,1984 vs 1776 (Alex) 🇺🇸,1485.242961,108
10,22311924,1984 vs 1776 🇺🇸,253.048930,108
15,755930626566389760,F* Your Short Memory 🏴 Anti-Cult of Absurdity,3419.837536,95
17,1463973734998585344,Babe Perales,7529.905456,86
21,722970901516427264,Katarzyna Ski 1776,177.522990,81


In [42]:
result2df.sort_values(by =['influence_score'],ascending = False).head(10)

,user_id,username,influence_score,total_retweeted
4,958729147408310272,James Finn,11492.967522,159
17,1463973734998585344,Babe Perales,7529.905456,86
478,138418838,Stigmabase | ORG,5167.500000,27
40,4282369753,Marci Maier,4488.000000,66
24,1427078665695756291,UNDERCOVER MOTHER #ExposeNAIS,4032.242012,79
415,1181952375399092225,Carl Hilberg,3623.699063,29
15,755930626566389760,F* Your Short Memory 🏴 Anti-Cult of Absurdity,3419.837536,95
119,59047876,Soph,3019.082171,45
30,3419594272,Dan Kleinman of SafeLibraries® 🟧,2657.856797,74
365,50626909,James Cooper,2636.303662,30
